In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [11]:
class denseLayer(nn.Module):
    def __init__(self, channels):
        super().__init__()

        self.bn1 = nn.BatchNorm2d(channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(channels, 128, kernel_size=1 ,stride=1, bias=False)
        self.bn2 = nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(128, 32, kernel_size=3 ,stride=1, padding=1, bias=False)

    def forward(self, x):
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.conv1(x)
        x = self.bn2(x)
        x = self.relu2(x)
        out = self.conv2(x)

        return out

In [12]:
class denseBlock(nn.Module):
    def __init__(self, channels, layer_count):
        super().__init__()
                
        self.layers = nn.ModuleList([denseLayer(channels + i*32) for i in range(layer_count)])
        
    def forward(self, x):
        layer_outputs = [x]
        # layer_outputs.append(x)
        
        for layers in self.layers:
            out = layers(x)
            layer_outputs.append(out)
            
            for output in layer_outputs:
                torch.cat([out, output], axis=1)

            x = out
            
        return x

In [16]:
class transition(nn.Module):
    def __init__(self, channels):
        super().__init__()

        self.bn = nn.BatchNorm2d(channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv2d(channels, channels//2, kernel_size=1, stride=1, bias=False)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2, padding=0)

        self.structure = nn.ModuleList([self.bn,
                                       self.relu,
                                       self.conv,
                                       self.pool])

    def forward(self, x):
        x = self.structure(x)

        return x


In [17]:
class densenet_121(nn.Module):
    def __init__(self):
        super().__init__()

        self.initial = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False),
        )
        
        self.num_layers = 4
        self.layer_count = [6, 12, 24, 16]
        self.channels = [64, 128, 256, 512]
        self.dense_block = nn.ModuleList()
        self.transitions = nn.ModuleList()

        for i in range(self.num_layers):
            self.dense_block.append(denseBlock(self.channels[i], self.layer_count[i]))
            if i < self.num_layers - 1:
                self.transitions.append(transition(self.channels[i+1]*2))
            

    def forward(self, x):
        x = self.initial(x)
        
        for i in range(self.num_layers):
            x = self.dense_block[i](x)
            if i < self.num_layers - 1:
                x = self.transition[i](x)
        
        return x

In [18]:
model = densenet_121()
print(model)

densenet_121(
  (initial): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (dense_block): ModuleList(
    (0): denseBlock(
      (layers): ModuleList(
        (0): denseLayer(
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (1): denseLayer(
          (bn1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_runn